<a href="https://colab.research.google.com/github/gannannavelorange/575/blob/master/src/ib_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
!git clone https://github.com/gannannavelorange/575

Cloning into '575'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 410 (delta 10), reused 0 (delta 0), pack-reused 382
Receiving objects: 100% (410/410), 1.90 MiB | 23.45 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [135]:
cd 575/src/data_IB_project

/content/575/src/data_IB_project/575/src/data_IB_project/575/src/data_IB_project/575/src/data_IB_project/575/src/data_IB_project


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import scipy.stats as ss
from scipy.stats import norm
from scipy.stats import t

In [0]:
'''
#get weekly interest rate change factor for coupon bond
def getdata(M,N):
  with open(M+'.csv') as file:
      reader=csv.reader(file)
      rows=[row for row in reader]
      DD=[row[1] for row in rows[1:293]]
      DD=[float(d)/100/2 for d in DD]
      D=[row[2] for row in rows[1+N:250+N]]
      D=[float(d)/100/2 for d in D]
      return DD,D
      #print(D)
      #print(D[0])


In [0]:
#get weekly interest rate change factor for coupon bond
def getdata(M,N):
  with open(M+'.csv') as file:
      reader=csv.reader(file)
      rows=[row for row in reader]
      DD=[row[1] for row in rows[1:62]]
      DD=[float(d)/100/2 for d in DD]
      D=[row[2] for row in rows[1+N:52+N]]
      D=[float(d)/100/2 for d in D]
      return DD,D

In [0]:
'''
g=getdata('20J28',0)[0]
print(g)
ks = norm.fit(g)
print(ks[0],ks[1])


In [0]:

# for each coupon, linloss could be calculated as L=-c*np.exp(r*t)(r*5-t*x) week=5 days
def linloss_bond(c,r,t,mu,sigma):
  exp=-c*np.exp(r*t)*(r*5-t*mu)
  #exp=-c*np.exp(r*t)*(r*1-t*mu)
  var=(np.exp(r*t)*t)**2*(sigma**2)
  return exp,var

In [0]:

def rolling(day,F,c,r,mu,sigma):
  Exp=-F*np.exp(r*(day/182+15))*(r*5-(day/182+15)*mu)
  Var=(np.exp(r*(day/182+15))*(day/182+15))**2*(sigma**2)
  for i in range(0,16):
    t=day/182+i
    Exp=Exp+linloss_bond(c,r,t,mu,sigma)[0]
    Var=Var+linloss_bond(c,r,t,mu,sigma)[1]
  return Exp,Var


In [0]:
c1=1.75/100*1000/2
F1=1000

In [0]:
for n in range(0,11):   # estimate 2010/02/07-2020/04/17
 dd=getdata('bond1',n)[0]
 d=getdata('bond1',n)[1]
 ks = norm.fit(d)
 r1=dd[50+n] 
 week=rolling(159-7*i,F1,c1,r1,ks[0],ks[1])   #152=2020/07/15-2020/02/07
 #print(week[0],week[1])
 print(week[0])


In [0]:
#put option

In [0]:
def N(x):
  cdf=norm.cdf(x,0,1)
  return cdf
def p(x):
  pdf=norm.pdf(x,0,1)
  return pdf

In [0]:
#get weekly stock price log return for option
def getdata2(M,N):
  with open(M+'.csv') as file:
      reader=csv.reader(file)
      rows=[row for row in reader]
      DD=[row[1] for row in rows[1:62]]
      DD=[float(d) for d in DD]
      D=[row[2] for row in rows[1+N:52+N]]
      D=[float(d) for d in D]
      return DD,D

In [152]:
djx=getdata2('DIA',0)
djx1=djx[0]
djx2=djx[1]
std=np.std(djx2)
print(djx1[50])


289.362


In [0]:
'''
def d(S,K,r,sigma,t):
 d1=(np.log(S/K)+(r+sigma**2/2)*t)/(sigma*np.sqrt(t))
 d2=d1-sigma*np.sqrt(t)
 return d1,d2

In [0]:
def deltaput(S,K,r,sigma,t):
  d1=(np.log(S/K)+(r+sigma**2/2)*t)/(sigma*np.sqrt(t))
  delta_P=N(float(d1))-1
  return delta_P

In [155]:
for i in range(0,11):
  djx=getdata2('DIA',i)
  djx1=djx[0]
  djx2=djx[1]
  std=np.std(djx2)
  delta=deltaput(djx1[50+i],293,0.02,std,(132-7*i)/250)   #t=(2020/06/18-2020/02/07)/250
  print(delta)


-0.5678012068436042
-0.20025717175127555
-0.6908078200144394
-1.0
-0.9999999999999969
-1.0
-1.0
-1.0
-1.0
-1.0
-0.9999999999999999
